In [2]:
import tensorflow as tf
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pickle
from dataShaping import *
from savePerf import *
import scipy.io.wavfile
import time
from tensorflow.python.tools import freeze_graph
from tensorflow.keras import layers
import tensorflow.keras as kr
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pyaudio
import wave
import sys


json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.compat.v1.keras.models.model_from_json(loaded_model_json)
loaded_model.summary()

num_step = 15 
features = 1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
_________________________________________________________________


Using TensorFlow backend.


In [ ]:
#wf = wave.open(sys.argv[1], 'rb') #read audio from audio device
CHUNK = 1024
form_1 = pyaudio.paInt16 # 16-bit resolution
chans = 2 # 2 channel
samp_rate = 44100 # 44.1kHz sampling rate
chunk = 8192 # 2^12 samples for buffer
dev_index = 2 # device index found by p.get_device_info_by_index(ii)
p = pyaudio.PyAudio()
stream = p.open(format = form_1,rate = samp_rate,channels = chans, \
                    input_device_index = dev_index,input = True, \
                    frames_per_buffer=chunk)
data = wf.readframes(CHUNK)
data=shapeDataRealTime(data,num_step,features)
data = loaded_model.predict(data)
while len(data) > 0:
    stream.write(data)
    data = wf.readframes(CHUNK)

# stop stream (4)
stream.stop_stream()
stream.close()

# close PyAudio (5)
p.terminate()